In [31]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import jax.numpy as jnp # Using jax.numpy for consistency if needed, but np is fine too
import os
import matplotlib.pyplot as plt # For potential plotting if scores are visualized
import csv

In [32]:
reference_trajectory_file = '/path to /dmp_ref.npy' 

loaded_data = np.load(reference_trajectory_file, allow_pickle=True)

reference_trajectory= loaded_data  


In [63]:
import numpy as np
import torch


def smoothness_loss(pred):
   
    dx = torch.diff(pred[:, :, 0], dim=0)
    dy = torch.diff(pred[:, :, 1], dim=0)
    d1 = dx**2 + dy**2
    loss_d1 = torch.sum(d1)
    
    ddx = torch.diff(dx, dim=0)
    ddy = torch.diff(dy, dim=0)
    d2 = ddx**2 + ddy**2
    loss_d2 = torch.sum(d2)
    
    dddx = torch.diff(ddx, dim=0)
    dddy = torch.diff(ddy, dim=0)
    d3 = dddx**2 + dddy**2
    loss_d3 = torch.sum(d3)
    
    loss = 0.2 * loss_d1 + 0.3 * loss_d2 + 0.5 * loss_d3
    
    return loss
 
def closed_shape_loss(pred):
    
    return torch.norm(pred[:,:,0] - pred[:,:,-1])
 
def symmetry_loss(pred):

    pred_squeezed = pred.squeeze(1) 
    centroid = torch.mean(pred_squeezed, dim=0)
    reflected_x = 2 * centroid[0] - pred_squeezed[:, 0]
    reflected_points = torch.stack((reflected_x, pred_squeezed[:, 1]), dim=1)
    
    diff = reflected_points[:, None, :] - pred_squeezed[None, :, :]
    dists = torch.norm(diff, dim=2)
    min_dists, _ = torch.min(dists, dim=1)
    
    return torch.mean(min_dists)
 
def shape_preservation_loss(pred, dmp_trajectory, weight_edge=1):

    dmp_tensor_unsqueezed = dmp_trajectory.unsqueeze(1)
    diffs = torch.norm(pred - dmp_tensor_unsqueezed, dim=-1)
    diffs = diffs.squeeze(1)
    
    weights = torch.ones_like(diffs)
    if diffs.shape[0] > 0:
        weights[0] = weight_edge
        if diffs.shape[0] > 1:
            weights[-1] = weight_edge
    
    loss = torch.sum(weights * diffs)
    return loss

def loss_to_score(loss_tensor, scaling_factor):
   
    return torch.exp(-loss_tensor * scaling_factor)


def scoring_function(trajectory_np, dmp_trajectory_np):
    
    pred = torch.tensor(trajectory_np, dtype=torch.float32).unsqueeze(1)
    dmp_trajectory = torch.tensor(dmp_trajectory_np, dtype=torch.float32)

    smooth_loss = smoothness_loss(pred)
    closed_loss = closed_shape_loss(pred)
    sym_loss = symmetry_loss(pred)
    shape_pres_loss = shape_preservation_loss(pred, dmp_trajectory)
    
    smooth_score = loss_to_score(smooth_loss, scaling_factor=0.01)
    closed_score = loss_to_score(closed_loss, scaling_factor=1.0)
    sym_score = loss_to_score(sym_loss, scaling_factor=1.0)
    shape_pres_score = loss_to_score(shape_pres_loss, scaling_factor=0.001)

    # Calculate the final score as a weighted average of the individual scores
    final_score = (0.03 * smooth_score +
                   0.03 * closed_score +
                   0.03 * sym_score +
                   0.9 * shape_pres_score)

    return {
        # Return the raw losses
        'smoothness_loss': smooth_loss.item(),
        'closed_shape_loss': closed_loss.item(),
        'symmetry_loss': sym_loss.item(),
        'shape_preservation_loss': shape_pres_loss.item(),
        
        # Return the single, normalized final score
        'final_score': final_score.item(),
    }

def process_rollouts(rollouts_file_path, dmp_ref_path):
    # 1. Load the rollout and DMP trajectories
    try:
        all_trajectories = np.load(rollouts_file_path, allow_pickle=True)
        if isinstance(all_trajectories, np.ndarray) and all_trajectories.ndim == 0:
            all_trajectories = all_trajectories.item()
        if isinstance(all_trajectories, dict):
            trajectories = all_trajectories.get('rollouts', [])
        elif isinstance(all_trajectories, (list, np.ndarray)):
            trajectories = all_trajectories
        else:
            raise TypeError("Loaded data is not a recognized trajectory format.")
    except FileNotFoundError:
        print(f"Error: The file '{rollouts_file_path}' was not found.")
        return None
    except Exception as e:
        print(f"An error occurred while loading the rollouts file: {e}")
        return None
    if not isinstance(trajectories, (list, np.ndarray)) or len(trajectories) == 0:
        print("No valid trajectories found to process.")
        return None
    
    # 2. Determine sequence_length and load DMP
    try:
        if isinstance(trajectories[0], dict):
            sequence_length = trajectories[0]['states'].shape[0]
        else:
            sequence_length = trajectories[0].shape[0]
        if sequence_length == 0:
            print("Trajectories are empty or have zero length.")
            return None
    except (IndexError, KeyError, AttributeError) as e:
        print(f"Error determining trajectory length from the first element: {e}")
        return None
    try:
        dmp_trajectory_np = np.load(dmp_ref_path)
        if dmp_trajectory_np.shape != (sequence_length, 2):
            raise ValueError(f"DMP trajectory shape {dmp_trajectory_np.shape} does not match rollout trajectory length {sequence_length}.")
        print(f"Loaded DMP reference trajectory of shape {dmp_trajectory_np.shape}")
    except FileNotFoundError:
        print(f"Error: The DMP reference file '{dmp_ref_path}' was not found.")
        return None
    except Exception as e:
        print(f"An error occurred while loading the DMP reference file: {e}")
        return None

    print("\n--- Processing trajectories (single pass) ---")
    all_results = []
    for i, trajectory in enumerate(trajectories):
        try:
            if isinstance(trajectory, dict):
                trajectory_np = np.asarray(trajectory['states'], dtype=np.float32)
            else:
                trajectory_np = np.asarray(trajectory, dtype=np.float32)
            if trajectory_np.shape != (sequence_length, 2):
                print(f"Warning: Trajectory {i+1} has unexpected shape {trajectory_np.shape}. Skipping.")
                continue
            score = scoring_function(trajectory_np, dmp_trajectory_np)
            all_results.append(score)
            print(f"Trajectory {i+1}/{len(trajectories)} processed. Results: {score}")
        except (KeyError, AttributeError) as e:
            print(f"Error processing trajectory {i+1} from a dictionary: {e}. Skipping.")
            continue
        except Exception as e:
            print(f"An unexpected error occurred while processing trajectory {i+1}: {e}. Skipping.")
            continue

    if not all_results:
        print("No trajectories were successfully processed.")
        return None

    all_keys = set().union(*(d.keys() for d in all_results))
    final_stats = {}
    for key in all_keys:
        values = [d[key] for d in all_results if key in d]
        if values:
            np_values = np.array(values)
            mean_value = np.mean(np_values)
            std_value = np.std(np_values)
            final_stats[f'{key}_mean'] = mean_value
            final_stats[f'{key}_std'] = std_value
    return final_stats

if __name__ == "__main__":
    rollouts_file_path = '/path to /cleaned rollouts.npy'
    dmp_ref_path = '/path to //dmp_ref.npy'

    results = process_rollouts(rollouts_file_path, dmp_ref_path)
    if results:
        print("\n--- Final Results ---")
        for key, value in results.items():
            print(f"{key:<30}: {value:.6f}")
    else:
        print("Could not generate final results.")

Loaded DMP reference trajectory of shape (60, 2)

--- Processing trajectories (single pass) ---
Trajectory 1/50 processed. Results: {'smoothness_loss': 0.42193442583084106, 'closed_shape_loss': 8.36997127532959, 'symmetry_loss': 0.19896899163722992, 'shape_preservation_loss': 20.547687530517578, 'final_score': 0.9361637234687805}
Trajectory 2/50 processed. Results: {'smoothness_loss': 0.556073009967804, 'closed_shape_loss': 11.758636474609375, 'symmetry_loss': 0.09529297798871994, 'shape_preservation_loss': 39.2220458984375, 'final_score': 0.9224905371665955}
Trajectory 3/50 processed. Results: {'smoothness_loss': 0.6836568117141724, 'closed_shape_loss': 12.190058708190918, 'symmetry_loss': 0.11868909746408463, 'shape_preservation_loss': 42.28376770019531, 'final_score': 0.9191762208938599}
Trajectory 4/50 processed. Results: {'smoothness_loss': 0.026881588622927666, 'closed_shape_loss': 4.99674129486084, 'symmetry_loss': 0.14042988419532776, 'shape_preservation_loss': 44.1830902099609

NOISY

In [64]:
import numpy as np
import torch

def smoothness_loss(pred):
    
    dx = torch.diff(pred[:, :, 0], dim=0)
    dy = torch.diff(pred[:, :, 1], dim=0)
    d1 = dx**2 + dy**2
    loss_d1 = torch.sum(d1)
    
    ddx = torch.diff(dx, dim=0)
    ddy = torch.diff(dy, dim=0)
    d2 = ddx**2 + ddy**2
    loss_d2 = torch.sum(d2)
    
    dddx = torch.diff(ddx, dim=0)
    dddy = torch.diff(ddy, dim=0)
    d3 = dddx**2 + dddy**2
    loss_d3 = torch.sum(d3)
    
    loss = 0.2 * loss_d1 + 0.3 * loss_d2 + 0.5 * loss_d3
    
    return loss
 
def closed_shape_loss(pred):
   
    return torch.norm(pred[:,:,0] - pred[:,:,-1])
 
def symmetry_loss(pred):
    
    pred_squeezed = pred.squeeze(1) 
    centroid = torch.mean(pred_squeezed, dim=0)
    reflected_x = 2 * centroid[0] - pred_squeezed[:, 0]
    reflected_points = torch.stack((reflected_x, pred_squeezed[:, 1]), dim=1)
    
    diff = reflected_points[:, None, :] - pred_squeezed[None, :, :]
    dists = torch.norm(diff, dim=2)
    min_dists, _ = torch.min(dists, dim=1)
    
    return torch.mean(min_dists)
 
def shape_preservation_loss(pred, dmp_trajectory, weight_edge=1):
    
    dmp_tensor_unsqueezed = dmp_trajectory.unsqueeze(1)
    diffs = torch.norm(pred - dmp_tensor_unsqueezed, dim=-1)
    diffs = diffs.squeeze(1)
    
    weights = torch.ones_like(diffs)
    if diffs.shape[0] > 0:
        weights[0] = weight_edge
        if diffs.shape[0] > 1:
            weights[-1] = weight_edge
    
    loss = torch.sum(weights * diffs)
    return loss

# --- Helper Function to Normalize Losses to Scores (Robust Method) ---

def loss_to_score(loss_tensor, scaling_factor):
    
    return torch.exp(-loss_tensor * scaling_factor)

# --- Scoring Function (Modified to return raw losses and one final score) ---

def scoring_function(trajectory_np, dmp_trajectory_np):
    
    pred = torch.tensor(trajectory_np, dtype=torch.float32).unsqueeze(1)
    dmp_trajectory = torch.tensor(dmp_trajectory_np, dtype=torch.float32)

    # 1. Calculate raw losses
    smooth_loss = smoothness_loss(pred)
    closed_loss = closed_shape_loss(pred)
    sym_loss = symmetry_loss(pred)
    shape_pres_loss = shape_preservation_loss(pred, dmp_trajectory)
    
    # 2. Convert raw losses to scores (0 to 1) for final score calculation
    smooth_score = loss_to_score(smooth_loss, scaling_factor=0.01)
    closed_score = loss_to_score(closed_loss, scaling_factor=1.0)
    sym_score = loss_to_score(sym_loss, scaling_factor=1.0)
    shape_pres_score = loss_to_score(shape_pres_loss, scaling_factor=0.001)

    # 3. Calculate the final score as a weighted average of the individual scores
    final_score = (0.03 * smooth_score +
                   0.03 * closed_score +
                   0.03 * sym_score +
                   0.9 * shape_pres_score)

    return {
        # Return the raw losses
        'smoothness_loss': smooth_loss.item(),
        'closed_shape_loss': closed_loss.item(),
        'symmetry_loss': sym_loss.item(),
        'shape_preservation_loss': shape_pres_loss.item(),
        
        # Return the single, normalized final score
        'final_score': final_score.item(),
    }

# --- Main processing logic ---

def process_rollouts(rollouts_file_path, dmp_ref_path):
   
    # 1. Load the rollout and DMP trajectories
    try:
        all_trajectories = np.load(rollouts_file_path, allow_pickle=True)
        if isinstance(all_trajectories, np.ndarray) and all_trajectories.ndim == 0:
            all_trajectories = all_trajectories.item()
        if isinstance(all_trajectories, dict):
            trajectories = all_trajectories.get('rollouts', [])
        elif isinstance(all_trajectories, (list, np.ndarray)):
            trajectories = all_trajectories
        else:
            raise TypeError("Loaded data is not a recognized trajectory format.")
    except FileNotFoundError:
        print(f"Error: The file '{rollouts_file_path}' was not found.")
        return None
    except Exception as e:
        print(f"An error occurred while loading the rollouts file: {e}")
        return None
    if not isinstance(trajectories, (list, np.ndarray)) or len(trajectories) == 0:
        print("No valid trajectories found to process.")
        return None
    
    # 2. Determine sequence_length and load DMP
    try:
        if isinstance(trajectories[0], dict):
            sequence_length = trajectories[0]['states'].shape[0]
        else:
            sequence_length = trajectories[0].shape[0]
        if sequence_length == 0:
            print("Trajectories are empty or have zero length.")
            return None
    except (IndexError, KeyError, AttributeError) as e:
        print(f"Error determining trajectory length from the first element: {e}")
        return None
    try:
        dmp_trajectory_np = np.load(dmp_ref_path)
        if dmp_trajectory_np.shape != (sequence_length, 2):
            raise ValueError(f"DMP trajectory shape {dmp_trajectory_np.shape} does not match rollout trajectory length {sequence_length}.")
        print(f"Loaded DMP reference trajectory of shape {dmp_trajectory_np.shape}")
    except FileNotFoundError:
        print(f"Error: The DMP reference file '{dmp_ref_path}' was not found.")
        return None
    except Exception as e:
        print(f"An error occurred while loading the DMP reference file: {e}")
        return None

    # --- Single Pass: Compute Scores and Collect Results ---
    print("\n--- Processing trajectories (single pass) ---")
    all_results = []
    for i, trajectory in enumerate(trajectories):
        try:
            if isinstance(trajectory, dict):
                trajectory_np = np.asarray(trajectory['states'], dtype=np.float32)
            else:
                trajectory_np = np.asarray(trajectory, dtype=np.float32)
            if trajectory_np.shape != (sequence_length, 2):
                print(f"Warning: Trajectory {i+1} has unexpected shape {trajectory_np.shape}. Skipping.")
                continue
            score = scoring_function(trajectory_np, dmp_trajectory_np)
            all_results.append(score)
            print(f"Trajectory {i+1}/{len(trajectories)} processed. Results: {score}")
        except (KeyError, AttributeError) as e:
            print(f"Error processing trajectory {i+1} from a dictionary: {e}. Skipping.")
            continue
        except Exception as e:
            print(f"An unexpected error occurred while processing trajectory {i+1}: {e}. Skipping.")
            continue

    if not all_results:
        print("No trajectories were successfully processed.")
        return None

    all_keys = set().union(*(d.keys() for d in all_results))
    final_stats = {}
    for key in all_keys:
        values = [d[key] for d in all_results if key in d]
        if values:
            np_values = np.array(values)
            mean_value = np.mean(np_values)
            std_value = np.std(np_values)
            final_stats[f'{key}_mean'] = mean_value
            final_stats[f'{key}_std'] = std_value
    return final_stats


if __name__ == "__main__":
    rollouts_file_path = '/path/to/noisy rollouts.npy'
    dmp_ref_path = '/path to/dmp_ref.npy'

    results = process_rollouts(rollouts_file_path, dmp_ref_path)
    if results:
        print("\n--- Final Results ---")
        for key, value in results.items():
            print(f"{key:<30}: {value:.6f}")
    else:
        print("Could not generate final results.")

Loaded DMP reference trajectory of shape (60, 2)

--- Processing trajectories (single pass) ---
Trajectory 1/50 processed. Results: {'smoothness_loss': 0.254970908164978, 'closed_shape_loss': 12.832009315490723, 'symmetry_loss': 0.7702195048332214, 'shape_preservation_loss': 117.12025451660156, 'final_score': 0.8443413972854614}
Trajectory 2/50 processed. Results: {'smoothness_loss': 0.1480376422405243, 'closed_shape_loss': 10.889790534973145, 'symmetry_loss': 0.4178933799266815, 'shape_preservation_loss': 112.12327575683594, 'final_score': 0.8542497754096985}
Trajectory 3/50 processed. Results: {'smoothness_loss': 0.3829572796821594, 'closed_shape_loss': 13.089766502380371, 'symmetry_loss': 0.4208792746067047, 'shape_preservation_loss': 115.92396545410156, 'final_score': 0.8510681390762329}
Trajectory 4/50 processed. Results: {'smoothness_loss': 0.48987966775894165, 'closed_shape_loss': 12.73538589477539, 'symmetry_loss': 1.2576298713684082, 'shape_preservation_loss': 117.201858520507